In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing, model_selection
from sklearn.linear_model import LinearRegression


In [ ]:
df = pd.read_csv('../input/absenteeism-dataset/MFGEmployees4.csv')

# Data Exploration

In [ ]:
df.head()

In [ ]:
# View Stats on Continuous variables
df.describe()

In [ ]:
# % NULL by column
df.isna().sum()

In [ ]:
# Create count dataframes for string/categorical variables
for col in df.columns:
    dfname = col+'_counts_df'
    globals()[dfname]= pd.DataFrame(df.value_counts([col]))
    
list(globals())

In [ ]:
Gender_counts_df, City_counts_df, DepartmentName_counts_df, BusinessUnit_counts_df

In [ ]:
City_counts_df

In [ ]:
Gender_counts_df.index.names[0]

In [ ]:
list(Gender_counts_df.transpose().columns.get_level_values(0))

In [ ]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [ ]:
# Show scatterplots on variable of interest (Absenteeism)
df.columns

In [ ]:
_ = plt.scatter('LengthService','AbsentHours', data=df)
_ = plt.xlabel('Length of Service')
_ = plt.ylabel('Hours Absent')

plt.show()

In [ ]:
_ = plt.scatter('Age','AbsentHours', data=df)
_ = plt.xlabel('Age')
_ = plt.ylabel('Hours Absent')

plt.show()

In [ ]:
print(max(df.Age), min(df.Age)) # Age of 3 - something is wrong here?!

In [ ]:
y_pos = list(Gender_counts_df.transpose().columns.get_level_values(0))
counts = list(Gender_counts_df[0])

_ = plt.barh(y_pos, counts)
_ = plt.xlabel('Gender Counts')
_ = plt.ylabel('Gender')
plt.show()

In [ ]:
vizList = ['Gender_counts_df',
          'JobTitle_counts_df',
           'BusinessUnit_counts_df',
           'City_counts_df',
           'Division_counts_df',
           'StoreLocation_counts_df'
          ]



for i in vizList:
    
    df_ = globals()[i].sort_values([0],ascending = False).head(10)
    ylab = df_.index.names[0]
    
    bars = list(df_.transpose().columns.get_level_values(0))
    counts = list(df_[0])
    
    plt.figure(figsize=(10,10))
    plotname = str([i])+"_plot"
    globals()[plotname] = plt.barh(bars, counts)
    globals()[plotname] = plt.xlabel('Counts')
    globals()[plotname] = plt.ylabel(ylab)
    plt.show()
   


In [ ]:
plt.hist(df['Age']) # Looks normally distributed

In [ ]:
#df['AbsentHours'].hist(color = 'blue')
df['AbsentHours'].apply(np.log1p).hist(color = 'orange')
plt.show

In [ ]:
# One hot encode
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(df)

In [ ]:
df['AbsentHours']

In [ ]:
# Run Linear Regression for Absenteeism based on Age

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = df.sample(frac = 0.75)

In [ ]:
X = pd.DataFrame({'EmployeeNumber':df['EmployeeNumber'],'Age':df['Age'].round(2)})
#X = np.append(X,np.zeros([len(X),0]),1)    #pd.Series([0 for x in range(len(X.index))], index=X.index)
# X = X.reshape(X.shape[1:])
y = df['AbsentHours']  #.apply(np.log1p).round(2)

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123)
X_train, y_train, X_test, y_test

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

# Plot outputs
plt.scatter(y_pred, y_test,  color='black', marker='.')
#plt.plot(y_pred, y_test, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

### Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(regr,X,y,cv=10)

In [ ]:
scores.mean(), scores.std()

## Repeating the same thing, but with more features

In [ ]:
X_2 = pd.DataFrame({'EmployeeNumber':df['EmployeeNumber'],
                  'Age':df['Age'].round(2), 
                  'LengthService':df['LengthService'],
                  'Gender':df['Gender']})

y_2 = df['AbsentHours']  #.apply(np.log1p).round(2)

In [ ]:
# Transform Male/Females to 1/0's
X_2['Gender'] = X_2['Gender'].replace({'M':1,'F':2})
X_2['Gender']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2, test_size=0.30, random_state=123)
X_train, y_train, X_test, y_test

In [ ]:
regr = linear_model.LinearRegression()
# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))

# Plot outputs
plt.scatter(y_pred, y_test,  color='black', marker='.')
#plt.plot(y_pred, y_test, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()


In [ ]:
scores_2 = cross_val_score(regr,X_2,y_2,cv=10)
scores_2.mean(), scores_2.std()

Our model slightly improves (lower MSE and higher accuracy) by adding in Length of Service and Gender, but our standard deviation is also a little higher.


Let's try the same idea, but with boosting.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb_reg = GradientBoostingRegressor(random_state=0)

gb_reg.fit(X_train,y_train)

y_pred = gb_reg.predict(X_test)


# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_pred, y_test))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_pred, y_test))



gb_reg.score(X_test, y_test)

In [ ]:
scores_3 = cross_val_score(gb_reg,X_2,y_2,cv=10)
scores_3.mean(), scores_3.std()

Again, our score improved. There is still more work to be done (One Hot Encoding BU's, Divisions, Departments) to try and improve the model, but using the GBR seems to be a strong fit! 

# Reading back in the dataset to for Validation Data

In [ ]:
df = pd.read_csv('../input/absenteeism-dataset/MFGEmployees4.csv')

In [ ]:
employeenums = X['EmployeeNumber']
employeenums.head()

In [ ]:
df = df[~df.EmployeeNumber.isin(employeenums)]

In [ ]:
df.shape

In [ ]:
df.head()